In [ ]:
import mmcv
from mmcv import Config
import open3d
import numpy as np
import torch
import MinkowskiEngine as ME
import numpy as np
from torch import nn

points = np.random.random(size=(1000,3))

tt = torch.tensor(points,dtype=torch.float)
tt.type(torch.float)[9]

In [ ]:
t1 = torch.randn(2,4)
t2 = torch.randn(2,4)
t3 = torch.randn(1,4)
pdist = nn.PairwiseDistance(2,keepdim= True)
criterion = nn.MSELoss()

In [ ]:
t3 =t1[0]-t2[0]
t3.abs().sum()

In [ ]:
t1[0,1].data.item()

In [ ]:
r1 = pdist(t1,t2)#.sum().cuda()
#r2 = pdist(t1,t3).sum().cuda()
#r3 = pdist(t3,t2).sum().cuda()

In [ ]:
import os
os.path.join("pcd_dir" ,"seq_idx" ,"2")

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read("/media/fangxu/Disk4T/fangxuPrj/MinkLoc3D/config/config_baseline.txt")
params = config['DEFAULT']
num_points = params.getint('num_points', 4096)


In [ ]:
params = config['TRAIN']
num_workers = params.getint('num_workers',7)
dataset_folder = params.get ('seq_train_list')
print(list(dataset_folder))
print(num_workers )
print(type(dataset_folder))

In [ ]:
import cv2
import os

cv2.__version__

In [ ]:
path = "/media/fangxu/Disk4T/fangxuPrj/Depth_Point_Location_Attention/tools/frame-000000.color.png"
Img = cv2.imread(path,cv2.CV_8UC1)
surf = cv2.xfeatures2d.SURF_create(400)
keypoints, descriptors = surf.detectAndCompute(Img, None)
print(len(keypoints))

In [ ]:
import torch
torch.__version__

In [ ]:
from torchvision import transforms
from PIL import Image
import numpy as np
path = "/media/fangxu/Disk4T/fangxuPrj/Depth_Point_Location_Attention/tools/frame-000000.depth.png"
transform_depth = transforms.Compose([
          transforms.Resize(256),
        transforms.CenterCrop(224),
                               #transforms.ToTensor(),
                               # transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                             ])

img_depth = Image.open( path).convert('I')
#img_depth =np.array(img_depth)#[np.newaxis,:]
img_depth= transform_depth(img_depth)
#img_depth.shape

In [ ]:
import numpy as np
import torch

In [ ]:
a = np.random.randn(3,3,1,1)

In [ ]:
b = torch.tensor(a)
b.squeeze().shape

In [ ]:
torch.float

In [ ]:
import torch, numpy
a=numpy.arange(5.)
a.flags.writeable=False
t=torch.from_numpy(a)

In [ ]:
from torchvision import transforms, models, datasets

In [ ]:
base_model = models.resnet34(pretrained=True)
base_model.fc.in_features

In [1]:
import os, sys
sys.path.append("/media/fangxu/Disk4T/fangxuPrj/Depth_Point_Location_Attention")
from dataset import make_dataloaders
from model import Fuse_PPNet, Pose_Depth_Net, standard_pose_loss, Pose_Pcd_Net
from mmcv import Config
import torch
import torch.nn.functional as F
from utils import median, quaternion_angular_error
from torch import nn
torch.backends.cudnn.enable =True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

config_path = '/media/fangxu/Disk4T/fangxuPrj/Depth_Point_Location_Attention/config/conf_pcd.py'

config = Config.fromfile(config_path)
pdist = nn.PairwiseDistance(2)

train_loader , test_loader = make_dataloaders(config)
#model = Fuse_PPNet(config)
model = Pose_Pcd_Net(config)
model = model.to(device)

# if config.pretrain_weight is not None:
#     model.load_state_dict( torch.load(config.pretrain_weight) )
#     print("load pretrain weight")

In [2]:
depth_base , pcd_base, base_t,base_q = enumerate(test_loader).__next__()[1]

In [3]:
#pcd_input = {e: pcd_base[e].to(device) for e in pcd_base}
t_gt, q_gt = base_t.to(device) , base_q.to(device)
pcd_input = {e: pcd_base[e].to(device) for e in pcd_base}


In [4]:
model.eval()
with torch.no_grad():
    dis_Err_Count, ort_Err_count = [], []
    t_infers, q_infers = model( pcd_input)

In [5]:
q_infers.cpu()

tensor([[-0.1825, -0.0234,  0.0481, -0.0044],
        [-0.1859, -0.0247,  0.0494, -0.0029],
        [-0.1841, -0.0263,  0.0500, -0.0054],
        [-0.1813, -0.0274,  0.0505, -0.0062],
        [-0.1847, -0.0250,  0.0502, -0.0051],
        [-0.1850, -0.0268,  0.0505, -0.0024],
        [-0.1829, -0.0253,  0.0481, -0.0041],
        [-0.1835, -0.0266,  0.0503, -0.0039],
        [-0.1853, -0.0262,  0.0496, -0.0045],
        [-0.1838, -0.0246,  0.0505, -0.0052]])

In [11]:
q_infer = F.normalize(q_infers, p=2, dim=1 )
ort_Err = quaternion_angular_error( q_infer, q_gt).cpu().numpy()

RuntimeError: CUDA error: device-side assert triggered

In [6]:
ort_Err

array([145.36958, 147.20853, 144.75111, 145.71577, 145.45235, 146.8012 ,
       146.92148, 145.63513, 145.85812, 143.8302 ], dtype=float32)

In [8]:
ddd = pdist(t_infers, t_gt)

In [16]:

#q_infer

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [4]:
import torch
a = torch.randn(10,3)
a.cuda()

tensor([[ 0.0554,  0.2591,  0.2918],
        [-0.3697,  0.2470, -0.4990],
        [-0.7781, -0.4152, -0.0231],
        [-1.4733, -2.1839, -0.0891],
        [-0.7738,  2.0468,  1.6853],
        [ 0.1338,  2.3174, -1.2279],
        [ 1.1289,  0.0024, -1.1528],
        [ 1.2758,  0.4467,  0.3057],
        [ 0.5126,  0.9566,  0.1841],
        [ 0.4284,  1.3931,  1.3314]], device='cuda:0')

In [8]:

q_infers= F.normalize(q_infers, p=2, dim=1 )
q_infers

#dis_Err = pdist(t_infer, t_gt).cpu().numpy()

tensor([[-0.2972, -0.7287,  0.4583, -0.4131],
        [-0.2901, -0.7481,  0.4422, -0.4008],
        [-0.2931, -0.7468,  0.4465, -0.3962],
        [-0.2851, -0.7474,  0.4478, -0.3994],
        [-0.2893, -0.7532,  0.4451, -0.3885],
        [-0.2791, -0.7572,  0.4504, -0.3820],
        [-0.2942, -0.7485,  0.4466, -0.3921],
        [-0.2903, -0.7530,  0.4355, -0.3988],
        [-0.2763, -0.7527,  0.4498, -0.3936],
        [-0.2809, -0.7504,  0.4454, -0.3996]])

In [14]:
dis_Err = pdist(t_infers, t_gt)
#dis_Err
#dis_Err_Count = dis_Err_Count + list(dis_Err)
#base_q.to(device)
dis_Err

In [ ]:
print(t_gt.shape)
print(t_infer.shape)

print(q_gt.shape)
print(q_infer.shape)

In [ ]:
q_infer = F.normalize(q_infer, p=2, dim=1 )
ort_Err = quaternion_angular_error( q_infer, q_gt).detach().cpu().numpy()

In [ ]:
q_infer

In [ ]:
torch.cuda.is_available()